In [1]:
# Based on https://www.youtube.com/watch?v=NxEHSAfFlK8 and https://github.com/AssemblyAI-Examples/Machine-Learning-From-Scratch/blob/main/04%20Decision%20Trees/DecisionTree.py
# This code implements a decision tree algorithm to predict a numerical feature
import numpy as np
import os
from numpy.random import choice
from collections import Counter

In [2]:
######## NODE CLASS #######
class Node:
    def __init__(self,left=None,right=None,threshold=None,feature=None,value=None):
        self.left      = left
        self.right     = right
        self.threshold = threshold
        self.feature   = feature
        self.value     = value
    def is_leaf(self): #checks if it is a leaf node
        if self.value == None:
            return False
        else:
            return True

In [15]:
####### DECISION CLASS #########
class Decision_tree:
    def __init__(self,max_depth=100,min_sample_split=2,nfeat=None):
        self.max_depth        = max_depth
        self.min_sample_split = min_sample_split
        self.nfeat            = nfeat
        self.root             = None
    def fit(self,X,y):
        self.nfeat = np.shape(X)[1] if self.nfeat is None else min(np.shape(X)[1],self.nfeat)
        self.root = self.grow_tree(X,y)
    def grow_tree(self,X,y,depth=0):
        #MAKE A PREDICTION IF CONDITIONS ARE MET
        nsamples,nfeatures = np.shape(X)
        if (nsamples < self.min_sample_split) or (depth >= self.max_depth) or (len(np.unique(y)) == 1):
            value = self.most_frequent_value(y)
            return Node(value=value)
        #SELECT THE FEATURE SUBSET TO BE CONSIDERED
        feat_subset = choice(nfeatures,self.nfeat,replace=False) #gettin a subset
        #EVALUATE IG FOR EACH CONFIG
        best_IG      = -1
        best_thresh  = None
        best_feature = None
        for feat in feat_subset:
            X_column   = X[:,feat]
            thresholds = np.unique(X_column)
            for thresh in thresholds:
                left_indx, right_indx = self.split_by_thresh(X_column,thresh)
                IG  = self.information_gain(y,left_indx,right_indx)
                if IG > best_IG:
                    best_IG      = IG
                    best_thresh  = thresh
                    best_feature = feat
        #EXECUTE THE SPLIT
        l_indx, r_indx = self.split_by_thresh(X[:,best_feature],best_thresh)
        #GROW_TREE LEFT AND RIGHT
        child_l = self.grow_tree(X[l_indx,:] ,y[l_indx] ,depth=depth+1)
        child_r = self.grow_tree(X[r_indx,:],y[r_indx],depth=depth+1)
        #RETURN THE NODE(LEFT,RIGHT)
        return Node(left=child_l,right=child_r,feature=best_feature,threshold=best_thresh)
    def most_frequent_value(self,y):
        return Counter(y).most_common()[0][0]
    def split_by_thresh(self,X_column,thresh):
        left_side  = np.argwhere(X_column <= thresh).flatten() #data that does not conforms with the threshold
        right_side = np.argwhere(X_column >  thresh).flatten()
        return left_side,right_side
    def information_gain(self,y,left_indx,right_indx):
        #calc parent's entropy
        par_en = self.entropy(y)
        #calculate childs' entropies
        y_l = y[left_indx]
        y_r = y[right_indx]
        if len(y_l) == 0 or len(y_r) == 0: # case the split is redundant
            return 0
        en_l = self.entropy(y_l)
        en_r = self.entropy(y_r)
        #calculate IG
        IG  = par_en - (len(y_l)/len(y))*en_l - (len(y_r)/len(y))*en_r
        return IG
    def entropy(self,y):
        pp = np.bincount(y)/len(y)
        return sum([-p*np.log(p) for p in pp if p > 0])
    def predict(self,X):
        return [self.check_tree_val(x,self.root) for x in X]
    def check_tree_val(self,x,node): #walks through the tree until it finds the leaf node
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.check_tree_val(x,node.left)
        return self.check_tree_val(x,node.right)

In [21]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

clf = Decision_tree(max_depth=200)
clf.fit(X_train, y_train)

ypred = clf.predict(X_test)
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test) #how many times the model get it right normalized by the size of pred

acc = accuracy(y_test, ypred)
print(acc)


0.9210526315789473
